In [ ]:
import pandas as pd
df = pd.read_csv('personality_datasert.csv')
df.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert
1,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert
2,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert
3,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert
4,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert


In [2]:
yes_no = {'Yes':1, 'No':0}
personality = {'Extrovert': 1, 'Introvert': 0}

In [ ]:
df = df.replace(to_replace={'Stage_fear': yes_no, 'Drained_after_socializing': yes_no, 'Personality': personality})